copy from makec2s.ipynb

In [7]:
import numpy as np
import random

# inspired by int ConnectionMatrix::genFlowBytes()
def genflowbytes():
    mean_bytes = 100.0 * 1024
    shape = 1.05
    scale = mean_bytes * (shape - 1)/shape

    x = np.random.exponential(scale=1.0/shape)
    flowbytes = int(scale * np.exp(x))
    return flowbytes

def adjustbytesbymtu(flowbytes):
  mss = 1500
  return mss * ((flowbytes+mss-1)//mss)





swlist = range(40,121,20)
# multlist = [[1,0,0],[1,0,0],[1,0,0],[1,0,0],[1,0,0]]
multlist = [[3,25,100],[1,65,100],[0,88,100],[0,63,100],[0,42,100]]
seed = 0
random.seed(seed)


for inumsw,numsw in enumerate(swlist):
  numleaf = int(numsw*0.8)
  numserver = (numsw//5)*(numsw//5)*12
  numport = int(numsw*0.8)
  c = int(numleaf*0.9)
  s = numleaf-c
  numspine = numsw-numleaf
  print(f"numsw={numsw},numleaf={numleaf},numspine={numspine},numport={numport},numserver={numserver}")
  mult = multlist[inumsw][0]
  numerator = multlist[inumsw][1]
  denominator = multlist[inumsw][2]
  
  # read leafspine serverfile
  serverfile = f"/home/annzhou/DRing/src/emp/datacentre/serverfiles/leafspine_{numserver}_{numsw}_{numport}"
  serverdict = dict()
  with open(serverfile,'r') as f:
      lines = f.readlines()
      for line in lines:
          tokens = line.split(',')
          serverdict[int(tokens[0])] = int(tokens[1])-numspine

  allfromswitches = list()
  alltoswitches = list()
  if c==numleaf:
    allfromswitches = range(numleaf)
  else:
    while len(allfromswitches)<c:
      rsw = random.randint(0,numleaf-1)
      if rsw not in allfromswitches:
        allfromswitches.append(rsw)
  # if s==numleaf:
  #   alltoswitches = range(numleaf)
  # else:
  #   while len(alltoswitches)<s:
  #     rsw = random.randint(0,numleaf-1)
  #     if rsw not in alltoswitches:
  #       alltoswitches.append(rsw)
  for mysw in range(numleaf):
    if mysw not in allfromswitches:
      alltoswitches.append(mysw)

  print(allfromswitches)
  print(alltoswitches)

  allfromservers = list()
  alltoservers = list()
  for lsserver in range(numserver):
    lsswitch = serverdict[lsserver]
    if lsswitch in allfromswitches:
      allfromservers.append(lsserver)
    if lsswitch in alltoswitches:
      alltoservers.append(lsserver)

  large_flow_threshold = 10 * 1024 * 1024
  base_flows = list()
  for fromsvr in allfromservers:
    for tosvr in alltoservers:
      if fromsvr!=tosvr:
        mybytes = genflowbytes()
        while mybytes<0 or mybytes>large_flow_threshold:
          mybytes = genflowbytes()
        mybytes = adjustbytesbymtu(mybytes)
        base_flows.append([fromsvr,tosvr,mybytes])

  flowfile = f"/home/annzhou/DRing/src/emp/datacentre/flowfiles/c2s_{numserver}_{numsw}_{numport}_{c}_{s}_{mult}_{numerator}_{denominator}_{seed}"
  with open(flowfile,'w') as f:
    if mult>0:
      for m in range(mult):
        for flow in base_flows:
          f.write(f"{flow[0]},{flow[1]},{flow[2]}\n")

    if denominator>0:
      for flow in base_flows:
        r = random.randint(0,denominator-1) # inclusive
        if r<numerator:
          f.write(f"{flow[0]},{flow[1]},{flow[2]}\n")

numsw=40,numleaf=32,numspine=8,numport=32,numserver=768
[24, 26, 2, 16, 31, 25, 19, 30, 22, 13, 8, 18, 6, 9, 4, 21, 27, 20, 28, 3, 0, 5, 15, 12, 14, 7, 11, 17]
[1, 10, 23, 29]
numsw=60,numleaf=48,numspine=12,numport=48,numserver=1728
[6, 18, 16, 3, 34, 27, 12, 29, 23, 40, 38, 35, 24, 21, 10, 8, 30, 32, 7, 9, 20, 45, 46, 17, 19, 31, 28, 39, 36, 22, 37, 33, 14, 15, 47, 5, 1, 26, 11, 42, 43, 13, 25]
[0, 2, 4, 41, 44]
numsw=80,numleaf=64,numspine=16,numport=64,numserver=3072
[7, 29, 10, 27, 11, 3, 37, 49, 55, 61, 4, 8, 35, 47, 1, 18, 22, 24, 30, 14, 53, 38, 6, 60, 51, 26, 63, 48, 43, 59, 54, 32, 17, 9, 52, 28, 23, 19, 13, 15, 62, 16, 42, 20, 5, 45, 41, 46, 25, 44, 0, 34, 56, 58, 31, 21, 12]
[2, 33, 36, 39, 40, 50, 57]
numsw=100,numleaf=80,numspine=20,numport=80,numserver=4800
[78, 46, 43, 42, 21, 19, 30, 58, 62, 74, 65, 67, 49, 61, 14, 4, 44, 54, 70, 23, 22, 77, 31, 79, 68, 40, 52, 11, 33, 17, 72, 63, 64, 1, 45, 12, 47, 3, 71, 37, 18, 15, 5, 56, 28, 6, 2, 9, 39, 48, 60, 76, 26, 13, 53, 25,